<a href="https://colab.research.google.com/github/Luxadevi/Ollama-Colab-intergration/blob/main/Ollama_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OLLAMA GOOGLE COLAB VERSION

Build to run ollama for no resource cost and have it accesed publicly

For this we need nat-tunneling running on a publicly exposed server

This script includes

* Downloading the newest cuda drivers
* Installing pciutils to show GPU
* Installing ollama
* Setting up a tunnel to acces te API
* Faking a port being in use for tunnel
* Downloading Modelfile to test with, location /content/



**IMPORTANT: nat-tunnel needs a port in use to check against**



In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
!mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/12.3.0/local_installers/cuda-repo-ubuntu2204-12-3-local_12.3.0-545.23.06-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2204-12-3-local_12.3.0-545.23.06-1_amd64.deb
!cp /var/cuda-repo-ubuntu2204-12-3-local/cuda-*-keyring.gpg /usr/share/keyrings/
!apt-get update
!apt-get -y install cuda-toolkit
!apt-get -y install cuda
!sudo apt install pciutils
!lspci
!wget https://ollama.ai/install.sh -O install.sh
!chmod +x install.sh
!./install.sh
!wget https://raw.githubusercontent.com/Luxadevi/Ollama-Colab-intergration/main/Modelfile
!git clone https://github.com/rofl0r/nat-tunnel.git
!wget https://raw.githubusercontent.com/Luxadevi/Ollama-Colab-intergration/main/portfaker.py





### Tunneling setup

* Download: https://github.com/rofl0r/nat-tunnel
* At least one port exposed for recieving connections

On your the server side (where you recieve the connection):

run ``python3 natsrv.py --mode server --secret s3cretP4ss --public 0.0.0.0:7000 --admin 0.0.0.0:8000``

Where **--secret** is the code you can set yourself and also will be prompted for in this notebook.

Port **7000** is the port that will be publicly available exposing ollama This is not neccesary per se if you want to acces it locally.

Port **8000** is the port where youll connect to so you need this one at least forwarded

# Tunneling
Start tunnel application and kill the python app that fakes a in use port

In [ ]:
!nohup python3 portfaker.py > app.log 2>&1 &

import os

user_ip = input("Enter your IP address to connect to: ")
user_secret = input("Enter the secret: ")

nats_command = f"python3 /content/nat-tunnel/natsrv.py --mode client --secret {user_secret} --local localhost:11434 --admin {user_ip}:8000 &"

# Run the nats_command
os.system(nats_command)
print("natsrv.py is running in the background.")

!pkill -f "python3 portfaker.py"

print("Succesful killed Fake port.")
print("Ready to launch ollama")

# Running ollama
You got 2 options or run ollama in the terminal output or have it running in the background. For debugging and the first stages of this app use the terminal output because its way easier to check whats happening in the back

In [ ]:
!OLLAMA_HOST=0.0.0.0:11434  OLLAMA_ORIGINS=http://0.0.0.0:* ollama serve

In [ ]:
import os

# Define the environment variables for ollama
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = 'http://0.0.0.0:*'

# Touch the output.log file to create it if it doesn't exist
touch_command = "touch output.log"
os.system(touch_command)

# Run the ollama serve command and log the stdout to output.log in the background
ollama_command = "nohup ollama serve > output.log 2>&1 &"
os.system(ollama_command)

print("ollama server is running in the background.")

To check status just run

In [ ]:
!curl localhost:11434

# Footnotes

You need the latest drivers and cuda toolkit
Usually for a http proxy certain applications will be used but i wanted to make this as easy as possible for the user.

A SSH SocksV5 proxy wouldn't work because of Google Colab restrictions.

You could however use a socks5 when you escape Colab so your API doesnt have to be exposed.

Ollama still closes after a while so keep tabs on it when it doesnt respond.